# Construction of the pMEC9001, 2 and 3 vectors

The vector pMEC1153 was descibed in [Romaní et al. 2014](http://www.sciencedirect.com/science/article/pii/S096085241401757X) where it was refered to as pMEC1049. The pMEC1153 expresses a D-xylose metabolic pathway and has a hygromycin selectable marker. The details of the construction of pMEC1153 can be found [here](pMEC1153.ipynb).

This document describe the construction of the pMEC9001, 2 and 3 vectors. The pMEC9001 is the pMEC1153 with an additional expression cassette for the Saccharomyces cerevisiae gene HAA1(YPR008W). The pMEC9002 is the pMEC1153 with an additional expression cassette for S. cerevisiae PRS3(YHL011C) and pMEC9003 has both of them.

| Vector   | Relevant property                                            |
|----------|--------------------------------------------------------------|
| pMEC9001 | [HAA1](http://www.yeastgenome.org/locus/S000006212/overview) |
| pMEC9002 | [PRS3](http://www.yeastgenome.org/locus/S000001003/overview) |
| pMEC9003 | HAA1 & PRS3                                                  |

Normally, this would be done following the yeast pathway kit strategy by adding genes with a set of new promoters and terminators, but in this case a requirement was to retain the native promoters and terminators for HAA1 and PRS3.

The strategy involves linearizing the vector in two locations (before and after the xylose pathway) and adding the HAA1 and PRS3 expression cassettes amplified using tailed primers.


## The PRS3 construct

The PRS3 cassette was previously cloned according to the description below in vector YEpJCP according to this description:

"obtained by PCR amplification of fragment carrying PRS3 from Saccharomyces cerevisiae CEN.PK113-7D genomic DNA using appropriate primers and insertion into plasmid pGEM-T Easy. Cloning into YEplac195KanMX using EcoRI digestion sites." [Cunha et al. 2015](http://www.sciencedirect.com/science/article/pii/S0960852415006707)

Primers:

    P1: TTATCTTCATCACCGCCATAC
    P2: ACAAGAGAAACTTTTGGGTAAAATG

The exact same PRS3 fragment will be cloned in pMEC9002.

In [51]:
import pydna

In [52]:
p1,p2 = pydna.parse_primers('''
>P1
TTATCTTCATCACCGCCATAC
>P2
ACAAGAGAAACTTTTGGGTAAAATG
''')

We gain access to the S. cerevisiae genome through the [pygenome](https://pypi.python.org/pypi/pygenome) module.

In [53]:
from pygenome import sg

In [54]:
PRS3_locus = pydna.Dseqrecord(sg.gene["PRS3"].locus())

The PRS3_locus contain the DNA from the end of the upstream ORF to the beginning of the downstream ORF.

In [55]:
PRS3_locus

Dseqrecord(-2963)

In [56]:
PRS3_product = pydna.pcr(p1, p2, PRS3_locus)

In [57]:
PRS3_product

Amplicon(1546)

In [58]:
PRS3_product.figure()

5TTATCTTCATCACCGCCATAC...CATTTTACCCAAAAGTTTCTCTTGT3
                         ||||||||||||||||||||||||| tm 55.7 (dbd) 63.4
                        3GTAAAATGGGTTTTCAAAGAGAACA5
5TTATCTTCATCACCGCCATAC3
 ||||||||||||||||||||| tm 55.5 (dbd) 62.9
3AATAGAAGTAGTGGCGGTATG...GTAAAATGGGTTTTCAAAGAGAACA5

The primers anneal perfectly to the template, so this is the PCR product we want.

# HAA1 construct

We will now do the same with the HAA1 cassette.

Vector: BHUM1737

Construction: obtained by PCR amplification of a SalI/BamHI fragment carrying HAA1 from yeast genomic DNA using appropriate primers and subsequent insertion into plasmid YEplac195.

Primers described in [Malcher et al. 2011](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3063667/) supporting information, Table S1

    HAA1hc_fw: GTC GAC CCC ATT TCC CCT TTC TTT TCC
    HAA1hc_rev: GGA TCC ATA CCT CAT CTC TGC GTG TTC G

In [59]:
h1,h2 = pydna.parse('''
>HAA1hc_fw
GTC GAC CCC ATT TCC CCT TTC TTT TCC
>HAA1hc_rev
GGA TCC ATA CCT CAT CTC TGC GTG TTC G
''')

In [60]:
HAA1_locus=pydna.Dseqrecord(sg.gene["HAA1"].locus())

In [61]:
HAA1_locus

Dseqrecord(-4085)

In [62]:
HAA1_product = pydna.pcr(h1, h2, HAA1_locus)

In [63]:
HAA1_product

Amplicon(3227)

In [64]:
HAA1_product.figure()

      5CCCATTTCCCCTTTCTTTTCC...CGAACACGCAGAGATGAGGTAT3
                               |||||||||||||||||||||| tm 59.9 (dbd) 66.7
                              3GCTTGTGCGTCTCTACTCCATACCTAGG5
5GTCGACCCCATTTCCCCTTTCTTTTCC3
       ||||||||||||||||||||| tm 57.0 (dbd) 67.6
      3GGGTAAAGGGGAAAGAAAAGG...GCTTGTGCGTCTCTACTCCATA5

These primers are tailed, but we have no reason to include these tails (containing restriction sites). We therefore cut six bp from the beginning and six bp from the end of the sequence:

In [65]:
HAA1_product = HAA1_product[6:-6]

In [66]:
HAA1_product.seq

Dseq(-3215)
CCCA..GTAT
GGGT..CATA

Now we will have to design tailed primers for the HAA1_product and the PRS3_product sequences so that we can add them to pMEC1153 by gap repair. First we have to decide with which restriction enzymes we should open the pMEC1153 vector.

The restriction enzymes below are candidates for linearizing the pMEC1153 before an after the cassette.

[XhoI](http://rebase.neb.com/rebase/enz/XhoI.html) [AleI](http://rebase.neb.com/rebase/enz/AleI.html) [OliI](http://rebase.neb.com/rebase/enz/OliI.html)

These enzymes are also unique in the pYPK0 based vectors, so we can use tha same strategy to create vectors expressing only the pRS3 and/or HAA1 but without the xylose pathway if needed.

In [67]:
pMEC1153 = pydna.read("pMEC1153.gb")

In [68]:
pMEC1153

File(id?)(o15488)

In [69]:
from Bio.Restriction import XhoI, AleI, OliI

In [70]:
pMEC1153_xho = pMEC1153.linearize(XhoI)

We design gap repair primers using the pydna assembly primers function

In [71]:
haa_f, haa_r = pydna.assembly_primers( (HAA1_product,), vector = pMEC1153_xho).pop()

In [72]:
haa_f.id = "haa_f"
haa_r.id = "haa_r"

In [73]:
print(haa_f.format("tab"))

haa_f	TGGTTTTACCGTGTGCGGAGATCAGGTTCTGATCCCCCATTTCCCCTTTCT



In [74]:
print(haa_r.format("tab"))

haa_r	AGACAAACCGTGGGACGAATTCTTAAGATGCTCGAATACCTCATCTCTGCGT



In [75]:
HAA1_recombination_product = pydna.pcr(haa_f, haa_r, HAA1_locus)

In [76]:
HAA1_recombination_product

Amplicon(3285)

In [77]:
HAA1_recombination_product.figure()

                                   5CCCATTTCCCCTTTCT...ACGCAGAGATGAGGTATT3
                                                       |||||||||||||||||| tm 53.5 (dbd) 56.4
                                                      3TGCGTCTCTACTCCATAAGCTCGTAGAATTCTTAAGCAGGGTGCCAAACAGA5
5TGGTTTTACCGTGTGCGGAGATCAGGTTCTGATCCCCCATTTCCCCTTTCT3
                                    |||||||||||||||| tm 51.1 (dbd) 59.1
                                   3GGGTAAAGGGGAAAGA...TGCGTCTCTACTCCATAA5

In [78]:
asm_haa1 = pydna.Assembly((pMEC1153_xho, HAA1_recombination_product))

In [79]:
asm_haa1

Assembly:
Sequences........................: [15492] [3285]
Sequences with shared homologies.: [15492] [3285]
Homology limit (bp)..............: 25
Number of overlaps...............: 2
Nodes in graph(incl. 5' & 3')....: 4
Only terminal overlaps...........: No
Circular products................: [18703]
Linear products..................: [18742] [18738] [39] [35]

In [80]:
candidate = asm_haa1.circular_products[0]

In [81]:
candidate.figure()

 -|pMEC1153|35
|           \/
|           /\
|           35|3285bp_PCR_prod|35
|                              \/
|                              /\
|                              35-
|                                 |
 ---------------------------------

In [82]:
pMEC9001 = candidate.synced(pMEC1153)

In [104]:
pMEC9001.stamp()

cSEGUID_BgBOlNv8lWGLExKGOpZa8J83H9I

In [105]:
pMEC9001.locus="pMEC9001"

The pMEC9001 is the pMEC1153 with HAA1. The sequence can be downloaded using the link below.

In [107]:
pMEC9001.write("pMEC9001.gb")

## PRS3

We will now make a pMEC1153 with PRS3 called pMEC9002.

In [85]:
pMEC1153_oli = pMEC1153.linearize(OliI)

The integration site was chosen to be the uniqie OliI site.

In [86]:
prs_f, prs_r = pydna.assembly_primers((PRS3_product,), vector=pMEC1153_oli).pop()

In [87]:
prs_f.id = "PRS3f"
prs_r.id = "PRS3r"

print(prs_f.format("tab"))
print(prs_r.format("tab"))

PRS3f	TAACGATGTAGTACAGCGTTTCCGCTTTTTCACCCTTATCTTCATCACCGC

PRS3r	CATAAGTACCCATCCAAGAGCACGCTTATTCACCAACAAGAGAAACTTTTGGG



In [88]:
PRS3_recombination_product = pydna.pcr(prs_f, prs_r, PRS3_locus)

In [89]:
PRS3_recombination_product

Amplicon(1616)

The recombination was designed for OliI but AleI was used.

In [90]:
pMEC1153_ale = pMEC1153.linearize(AleI)

In [91]:
asm_prs3 = pydna.Assembly((pMEC1153_ale, PRS3_recombination_product))

In [92]:
asm_prs3

Assembly:
Sequences........................: [15488] [1616]
Sequences with shared homologies.: [15488] [1616]
Homology limit (bp)..............: 25
Number of overlaps...............: 2
Nodes in graph(incl. 5' & 3')....: 4
Only terminal overlaps...........: No
Circular products................: [17034]
Linear products..................: [17069] [17069] [35] [35]

In [93]:
candidate = asm_prs3.circular_products[0]

In [94]:
candidate

Contig(o17034)

In [99]:
pMEC9002 = candidate.synced(pMEC1153)

In [100]:
pMEC9002.locus = "pMEC9002"

In [102]:
pMEC9002.stamp()

cSEGUID_5qldWd2jGxUyMnMKECLGpNvJv2M

The pMEC9002 vector is the pMEC1153 with PRS3

In [103]:
pMEC9002.write("pMEC9002.gb")

## pMEC9003

The HAA1 and PRS3 cassettes were added to the plasmid in one step to the plasmid digested with both XhoI and AleI. Cutting with XhoI and AleI makes two fragments about 6 and 9 kb.

In [108]:
pMEC1153_6kb, pMEC1153_9kbp = pMEC1153.cut(XhoI, AleI)

In [109]:
pMEC1153_6kb

Dseqrecord(-6017)

In [111]:
pMEC1153_9kbp

Dseqrecord(-9475)

In [112]:
pMEC1153_6kb.locus = "pMEC1153_6kb"
pMEC1153_9kbp.locus = "pMEC1153_9kbp"

In [113]:
asm_prs_haa = pydna.Assembly((pMEC1153_6kb, pMEC1153_9kbp, HAA1_recombination_product, PRS3_recombination_product))

In [114]:
asm_prs_haa

Assembly:
Sequences........................: [6017] [9475] [3285] [1616]
Sequences with shared homologies.: [6017] [3285] [1616] [9475]
Homology limit (bp)..............: 25
Number of overlaps...............: 4
Nodes in graph(incl. 5' & 3')....: 6
Only terminal overlaps...........: No
Circular products................: [20249]
Linear products..................: [20288] [20284] [20284] [20284] [18703] [17038] [14306] [12725] [11056] [10844] [9263] [7598] [39] [35] [35] [35]

In [115]:
candidate = asm_prs_haa.circular_products[0]

In [116]:
candidate

Contig(o20249)

In [117]:
pMEC9003 = candidate.synced(pMEC1153)

In [118]:
pMEC9003.stamp()

cSEGUID_hhymwO1hS1IXp9n4XX4eODGWuhU

In [119]:
pMEC9003.locus="pMEC9003"

pMEC9003 is the pMEC1153 with both HAA1 and PRS3. The sequence can be downloaded from the link below.

In [120]:
pMEC9003

Dseqrecord(o20249)

In [121]:
pMEC9003.write("pMEC9003.gb")